### Script for function to pull vitals data

This function loads chart events (specified by user-entered ID) into a dataframe, filters events by ventilataed patients, determines the last reading before extubation, takes the mean of this value for any duplicate readings (happens when multiple sensors used simultanrously) and returns the data as a column in the master dataframe

#### impot useful libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### user-defined variables that change every time

In [2]:
event = "pulseox"
data_dir = "../data/feathered/"
export_dir = "../data/processed/"

datafile = data_dir+event
mean_export_file = export_dir+event
std_export_file = export_dir+"std_"+event
meanColumns = ['hadm_id',event]
stdColumns = ['hadm_id','std_'+event]

In [3]:
df = pd.read_feather(datafile)
df['valuenum']= pd.to_numeric(df['valuenum'])

In [4]:
# to convert fahrenheit to celcius: (F-32) x 5/9
#df['valuenum'] = np.where(df['itemid']==223761,(df['valuenum']-32)*(5/9),df['valuenum'])

In [5]:
# convert height in cm to inches- divide by 2.74
#df['valuenum'] = np.where(df['itemid']==226730,(df['valuenum']/2.54),df['valuenum'])

In [6]:
df

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,endtime,re_intub_class,time_on_vent
0,10004235,24181354,30276431,2196-02-24 18:00:00,2196-02-24 18:14:00,220277,94,94.0,%,0,2196-02-27 16:28:00,0,71.600000
1,10004235,24181354,30276431,2196-02-24 19:00:00,2196-02-24 19:43:00,220277,99,99.0,%,0,2196-02-27 16:28:00,0,71.600000
2,10004235,24181354,30276431,2196-02-24 20:00:00,2196-02-24 19:57:00,220277,97,97.0,%,0,2196-02-27 16:28:00,0,71.600000
3,10004235,24181354,30276431,2196-02-24 21:00:00,2196-02-24 21:04:00,220277,98,98.0,%,0,2196-02-27 16:28:00,0,71.600000
4,10004235,24181354,30276431,2196-02-24 22:00:00,2196-02-24 22:29:00,220277,100,100.0,%,0,2196-02-27 16:28:00,0,71.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005896,19999068,21606769,31096823,2161-08-27 11:00:00,2161-08-27 11:51:00,220277,98,98.0,%,0,2161-08-28 13:36:00,0,70.016667
3005897,19999068,21606769,31096823,2161-08-27 12:00:00,2161-08-27 12:08:00,220277,100,100.0,%,0,2161-08-28 13:36:00,0,70.016667
3005898,19999068,21606769,31096823,2161-08-27 13:00:00,2161-08-27 13:38:00,220277,98,98.0,%,0,2161-08-28 13:36:00,0,70.016667
3005899,19999068,21606769,31096823,2161-08-27 14:01:00,2161-08-27 14:12:00,220277,100,100.0,%,0,2161-08-28 13:36:00,0,70.016667


#### Convert times to pandas datetime object, find the time difference between each event and the time of extubation ('endtime') and find the last time an event occurred before extubation

In [7]:
df['endtime'] = pd.to_datetime(df['endtime'])
df['charttime'] = pd.to_datetime(df['charttime'])
df['time_diff']= df['endtime']-df['charttime']

In [8]:
df['timediffhour'] = df['time_diff'] / np.timedelta64(1, 'h') # convert time to hours
df_pre = df.copy()
df_pre = df[df['timediffhour']>0 & (df['timediffhour']<2)]

In [9]:
df_pre['min']=df_pre.groupby('hadm_id')['timediffhour'].transform('min') # find the last reading 
            # time before extubation occured and place in a new channel ('min')

<ipython-input-9-1f0533c03414>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre['min']=df_pre.groupby('hadm_id')['timediffhour'].transform('min') # find the last reading


In [10]:
last_value_pre = df_pre[df_pre['timediffhour'] == df_pre['min']] 
            # select only rows where 'timediffhour' is the same as the minimum time ('min')

In [11]:
last_value_pre

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,endtime,re_intub_class,time_on_vent,time_diff,timediffhour,min
69,10004235,24181354,30276431,2196-02-27 16:00:00,2196-02-27 16:06:00,220277,100,100.0,%,0,2196-02-27 16:28:00,0,71.600000,0 days 00:28:00,0.466667,0.466667
149,10005348,25239799,31523640,2130-10-28 04:00:00,2130-10-28 04:04:00,220277,100,100.0,%,0,2130-10-28 04:35:00,0,11.833333,0 days 00:35:00,0.583333,0.583333
206,10019003,27525946,30460871,2153-04-14 17:00:00,2153-04-14 17:25:00,220277,95,95.0,%,0,2153-04-14 17:50:00,0,22.066667,0 days 00:50:00,0.833333,0.833333
611,10035631,29276678,35275147,2116-03-09 18:00:00,2116-03-09 18:03:00,220277,98,98.0,%,0,2116-03-09 18:11:00,0,217.983333,0 days 00:11:00,0.183333,0.183333
774,10035747,27083519,39236053,2126-05-15 12:14:00,2126-05-15 12:14:00,220277,100,100.0,%,0,2126-05-15 13:00:00,0,143.000000,0 days 00:46:00,0.766667,0.766667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005404,19965610,28545396,30691619,2125-09-28 12:00:00,2125-09-28 12:48:00,220277,94,94.0,%,0,2125-09-28 12:51:00,0,52.733333,0 days 00:51:00,0.850000,0.850000
3005520,19970491,22119205,30220330,2131-02-13 09:00:00,2131-02-13 09:21:00,220277,99,99.0,%,0,2131-02-13 09:17:00,0,48.733333,0 days 00:17:00,0.283333,0.283333
3005575,19970491,20897702,32207420,2131-05-11 21:20:00,2131-05-11 21:26:00,220277,100,100.0,%,0,2131-05-11 21:22:00,0,25.366667,0 days 00:02:00,0.033333,0.033333
3005727,19970491,29133530,38263194,2132-04-28 10:00:00,2132-04-28 10:01:00,220277,98,98.0,%,0,2132-04-28 12:19:00,0,90.316667,0 days 02:19:00,2.316667,2.316667


In [12]:
avg_dupValues = last_value_pre.groupby('hadm_id')['valuenum'].mean() # where duplicate values exist,take the mean
meanBP = avg_dupValues.reset_index()
meanBP.columns = (meanColumns)

In [13]:
svents = pd.read_csv(export_dir +'sample_vents.csv')
mean_df = pd.merge(left = svents, right=meanBP, how = 'left', left_on='hadm_id', right_on='hadm_id')   

In [14]:
mean_df

,hadm_id,endtime,time_on_vent,re_intub_class,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,hours_to_death,pulseox
0,28038802,2185-12-20 09:59:00,108.800000,0,18917458,2185-12-15 00:17:00,2185-12-22 17:15:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,MARRIED,WHITE,2185-12-14 23:05:00,2185-12-15 01:31:00,0,NaN,88.0
1,21790335,2140-03-11 13:44:00,33.000000,0,19704964,2140-03-09 14:44:00,2140-03-15 13:00:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,?,MARRIED,UNABLE TO OBTAIN,NaN,NaN,0,NaN,92.0
2,24357541,2177-02-10 14:00:00,659.416667,0,19004463,2176-12-16 18:11:00,2177-02-15 17:00:00,NaN,URGENT,TRANSFER FROM HOSPITAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0,NaN,94.0
3,22188993,2145-11-04 18:40:00,143.666667,0,11538389,2145-10-29 12:46:00,2145-11-09 17:30:00,2145-11-09 17:30:00,EW EMER.,EMERGENCY ROOM,DIED,Medicare,ENGLISH,MARRIED,UNKNOWN,2145-10-29 11:07:00,2145-10-29 14:00:00,1,-118.833333,98.0
4,21880799,2134-05-22 17:58:00,54.966667,0,16606203,2134-05-20 09:51:00,2134-05-22 18:00:00,NaN,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,?,MARRIED,HISPANIC/LATINO,2134-05-20 06:11:00,2134-05-20 11:15:00,0,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16399,29960248,2146-12-09 14:23:00,73.466667,1,16691924,2146-11-30 16:09:00,2146-12-17 17:04:00,NaN,DIRECT EMER.,CLINIC REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,WIDOWED,WHITE,NaN,NaN,0,NaN,96.0
16400,29962016,2135-10-30 11:45:00,218.083333,1,16796371,2135-10-21 08:16:00,2135-11-04 13:30:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2135-10-21 05:47:00,2135-10-21 09:45:00,0,NaN,100.0
16401,29974575,2131-03-03 19:54:00,98.900000,1,10020944,2131-02-27 15:34:00,2131-03-13 17:01:00,NaN,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,NaN,UNKNOWN,2131-02-27 13:16:00,2131-02-27 16:40:00,0,NaN,100.0
16402,29987115,2148-02-19 10:00:00,44.000000,1,19652570,2148-02-16 17:42:00,2148-02-20 19:00:00,2148-02-20 19:00:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,1,-33.000000,99.0


In [15]:
mean_df.to_feather(mean_export_file)

In [16]:
#num_ventilations = last_value_pre['hadm_id'].value_counts()
#svdf = last_value_pre[last_value_pre['hadm_id'].isin(num_ventilations.index[num_ventilations ==2])]
#sns.countplot(svdf['valuenum'])

### Getting variability over last 6 hours

In [17]:
df_var = df.copy()
df_var = df[(df['timediffhour']>0)&(df['timediffhour']<6)]

varBP = df_var.groupby('hadm_id')['valuenum'].std()
varBP = varBP.reset_index()
varBP.columns = (stdColumns)

In [18]:
#plt.hist(merged_inner_var['timediffhour'])

In [19]:
std_df = pd.merge(left = svents, right=varBP, how = 'left', left_on='hadm_id', right_on='hadm_id')

In [21]:
std_df[std_df['std_pulseox']==0]

,hadm_id,endtime,time_on_vent,re_intub_class,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,hours_to_death,std_pulseox
4,21880799,2134-05-22 17:58:00,54.966667,0,16606203,2134-05-20 09:51:00,2134-05-22 18:00:00,NaN,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,?,MARRIED,HISPANIC/LATINO,2134-05-20 06:11:00,2134-05-20 11:15:00,0,NaN,0.0
6,28691076,2149-08-01 15:31:00,25.983333,0,17102703,2149-07-31 12:46:00,2149-08-05 09:45:00,2149-08-05 09:45:00,EW EMER.,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,BLACK/AFRICAN AMERICAN,2149-07-31 11:35:00,2149-07-31 13:47:00,1,-90.233333,0.0
12,23481036,2112-05-25 09:52:00,83.866667,0,18558946,2112-05-21 12:46:00,2112-06-04 14:23:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,ENGLISH,MARRIED,WHITE,2112-05-21 09:23:00,2112-05-21 14:05:00,0,NaN,0.0
25,24874203,2130-10-02 06:40:00,6.650000,0,10789613,2130-10-01 16:35:00,2130-10-15 17:20:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,?,MARRIED,UNABLE TO OBTAIN,2130-10-01 15:55:00,2130-10-01 16:11:00,0,NaN,0.0
26,24517521,2131-02-14 17:22:00,17.866667,0,18174739,2131-02-13 23:24:00,2131-02-21 06:08:00,2131-02-21 06:08:00,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,WIDOWED,WHITE,2131-02-13 16:57:00,2131-02-14 00:53:00,1,-156.766667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16382,29834594,2193-12-21 14:35:00,8.666667,1,12564356,2193-12-21 01:48:00,2194-01-07 16:40:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Other,ENGLISH,SINGLE,WHITE,2193-12-21 00:45:00,2193-12-21 05:25:00,0,NaN,0.0
16390,29877257,2178-10-11 14:35:00,172.250000,1,12763939,2178-10-04 03:12:00,2178-11-07 16:30:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Other,ENGLISH,SINGLE,WHITE,2178-10-04 02:34:00,2178-10-04 03:26:00,0,NaN,0.0
16391,29885103,2120-05-19 17:40:00,34.583333,1,18770897,2120-05-17 21:56:00,2120-05-31 11:20:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Other,ENGLISH,NaN,OTHER,2120-05-17 19:21:00,2120-05-17 23:41:00,0,NaN,0.0
16398,29951431,2173-10-10 09:32:00,14.783333,1,18570301,2173-10-09 18:35:00,2173-10-26 17:35:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,ENGLISH,NaN,WHITE,2173-10-09 14:58:00,2173-10-09 19:05:00,0,NaN,0.0


In [22]:
df_pre[df_pre['hadm_id']==21880799]

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,endtime,re_intub_class,time_on_vent,time_diff,timediffhour,min
248132,16606203,21880799,30973201,2134-05-20 11:39:00,2134-05-20 11:41:00,220277,99,99.0,%,0,2134-05-22 17:58:00,0,54.966667,2 days 06:19:00,54.316667,1.966667
248133,16606203,21880799,30973201,2134-05-20 11:40:00,2134-05-20 11:41:00,220277,98,98.0,%,0,2134-05-22 17:58:00,0,54.966667,2 days 06:18:00,54.300000,1.966667
248134,16606203,21880799,30973201,2134-05-20 22:00:00,2134-05-20 22:29:00,220277,100,100.0,%,0,2134-05-22 17:58:00,0,54.966667,1 days 19:58:00,43.966667,1.966667
248135,16606203,21880799,30973201,2134-05-20 17:00:00,2134-05-20 17:41:00,220277,100,100.0,%,0,2134-05-22 17:58:00,0,54.966667,2 days 00:58:00,48.966667,1.966667
248136,16606203,21880799,30973201,2134-05-20 18:00:00,2134-05-20 18:52:00,220277,100,100.0,%,0,2134-05-22 17:58:00,0,54.966667,1 days 23:58:00,47.966667,1.966667
248137,16606203,21880799,30973201,2134-05-20 19:00:00,2134-05-20 20:18:00,220277,100,100.0,%,0,2134-05-22 17:58:00,0,54.966667,1 days 22:58:00,46.966667,1.966667
248138,16606203,21880799,30973201,2134-05-20 20:00:00,2134-05-20 20:18:00,220277,97,97.0,%,0,2134-05-22 17:58:00,0,54.966667,1 days 21:58:00,45.966667,1.966667
248139,16606203,21880799,30973201,2134-05-20 21:00:00,2134-05-20 21:22:00,220277,100,100.0,%,0,2134-05-22 17:58:00,0,54.966667,1 days 20:58:00,44.966667,1.966667
248140,16606203,21880799,30973201,2134-05-20 16:00:00,2134-05-20 18:52:00,220277,99,99.0,%,0,2134-05-22 17:58:00,0,54.966667,2 days 01:58:00,49.966667,1.966667
248141,16606203,21880799,30973201,2134-05-20 16:10:00,2134-05-20 16:10:00,220277,100,100.0,%,0,2134-05-22 17:58:00,0,54.966667,2 days 01:48:00,49.800000,1.966667


In [20]:
std_df.to_feather(std_export_file)

new_df = pd.merge(left = svents, right=meanBP, how = 'left', left_on='hadm_id', right_on='hadm_id')
new_df

new_df = pd.merge(left = new_df, right=varBP, how = 'left', left_on='hadm_id', right_on='hadm_id')
new_df

new_df.drop(['temp','stdTemp'],axis=1,inplace=True)

new_df.to_csv('feature_table.csv',index= False)